In [ ]:
'''
📌 Project Overview

This project implements a Crypto Model Context Protocol (MCP) Server using the CoinGecko API.
It provides a unified interface for multiple cryptocurrency data operations — all accessible via a single /mcp endpoint that accepts JSON-based requests.

The system is built using Flask (Python), and runs completely inside Google Colab with background threading to prevent blocking.
It demonstrates API integration, modular design, error handling, and basic test coverage

'''

# Simple MCP SERVER USING COINGECKO API

In [ ]:
# Step1  -  MCP server

In [1]:
from flask import Flask, request, jsonify
from threading import Thread
import requests

# ===== CONFIG =====
API_KEY = "CG-fhdRS442CWoiXmvCUHwUixWW"   # Demo key
BASE = "https://api.coingecko.com/api/v3"  # ✅ Demo API base

app = Flask(__name__)

def cg_get(path, params=None):
    headers = {"x-cg-pro-api-key": API_KEY}
    try:
        r = requests.get(f"{BASE}/{path}", params=params, headers=headers, timeout=10)
        if not r.ok:
            return {"error": f"HTTP {r.status_code}: {r.text}"}
        return r.json()
    except Exception as e:
        return {"error": str(e)}

@app.route("/mcp", methods=["POST"])
def handle_mcp():
    data = request.get_json(force=True)
    method = data.get("method")
    params = data.get("params", {})

    try:
        # === getPrice ===
        if method == "getPrice":
            symbol = params.get("symbol", "bitcoin")
            vs = params.get("vs_currency", "usd")
            query = {"vs_currencies": vs, "ids": symbol}
            res = cg_get("simple/price", query)
            if "error" in res:
                return jsonify(res), 500
            return jsonify({"result": res})

        # === getMarketCap ===
        elif method == "getMarketCap":
            symbol = params.get("symbol", "bitcoin")
            d = cg_get(f"coins/{symbol}")
            if "error" in d:
                return jsonify(d), 500
            res = {
                "id": d["id"],
                "market_cap_usd": d["market_data"]["market_cap"]["usd"],
                "rank": d["market_cap_rank"]
            }
            return jsonify({"result": res})

        # === getTopCoins ===
        elif method == "getTopCoins":
            limit = int(params.get("limit", 5))
            d = cg_get("coins/markets", {
                "vs_currency": "usd",
                "order": "market_cap_desc",
                "per_page": limit,
                "page": 1
            })
            if "error" in d:
                return jsonify(d), 500
            res = [{"id": c["id"], "symbol": c["symbol"], "price": c["current_price"]} for c in d]
            return jsonify({"result": res})

        # === getExchangeRate ===
        elif method == "getExchangeRate":
            f = params.get("from_currency", "bitcoin")
            t = params.get("to_currency", "ethereum")
            q = {"vs_currencies": "usd", "ids": f + "," + t}
            data = cg_get("simple/price", q)
            if "error" in data:
                return jsonify(data), 500
            if f not in data or t not in data:
                return jsonify({"error": "Invalid currencies"}), 400
            rate = data[f]["usd"] / data[t]["usd"]
            return jsonify({"result": {"from": f, "to": t, "exchange_rate": rate}})

        # === listTools ===
        elif method == "listTools":
            return jsonify({"result": [
                "getPrice", "getMarketCap", "getTopCoins", "getExchangeRate", "listTools"
            ]})

        else:
            return jsonify({"error": f"Unknown method {method}"}), 400

    except Exception as e:
        return jsonify({"error": str(e)}), 500


# === Run Flask in background ===
def run_flask():
    app.run(host="0.0.0.0", port=8000)

Thread(target=run_flask, daemon=True).start()
print("✅ MCP server (Demo) started on http://127.0.0.1:8000/mcp")


✅ MCP server (Demo) started on http://127.0.0.1:8000/mcp


In [ ]:
# CALLING OF THE SERVER WITH PORT

In [2]:
import requests, json
BASE_URL = "http://127.0.0.1:8000/mcp"

def call_mcp(method, params=None):
    r = requests.post(BASE_URL, json={"method": method, "params": params or {}})
    return r.json()

print("→ listTools")
print(call_mcp("listTools"))

print("\n→ getPrice")
print(json.dumps(call_mcp("getPrice", {"symbol":"bitcoin","vs_currency":"usd"}), indent=2))

print("\n→ getMarketCap")
print(json.dumps(call_mcp("getMarketCap", {"symbol":"bitcoin"}), indent=2))

print("\n→ getTopCoins")
print(json.dumps(call_mcp("getTopCoins", {"limit":3}), indent=2))

print("\n→ getExchangeRate BTC→ETH")
print(json.dumps(call_mcp("getExchangeRate", {"from_currency":"bitcoin","to_currency":"ethereum"}), indent=2))


INFO:werkzeug:127.0.0.1 - - [04/Oct/2025 17:11:10] "POST /mcp HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Oct/2025 17:11:10] "POST /mcp HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Oct/2025 17:11:10] "POST /mcp HTTP/1.1" 200 -


→ listTools
{'result': ['getPrice', 'getMarketCap', 'getTopCoins', 'getExchangeRate', 'listTools']}

→ getPrice
{
  "result": {
    "bitcoin": {
      "usd": 121877
    }
  }
}

→ getMarketCap
{
  "result": {
    "id": "bitcoin",
    "market_cap_usd": 2428795198671,
    "rank": 1
  }
}

→ getTopCoins


INFO:werkzeug:127.0.0.1 - - [04/Oct/2025 17:11:10] "POST /mcp HTTP/1.1" 500 -
INFO:werkzeug:127.0.0.1 - - [04/Oct/2025 17:11:10] "POST /mcp HTTP/1.1" 200 -


{
  "error": "HTTP 400: {\"timestamp\":\"2025-10-04T17:11:10.599+00:00\",\"error_code\":10010,\"status\":{\"error_message\":\"If you are using Pro API key, please change your root URL from api.coingecko.com to pro-api.coingecko.com  Please refer here for more details: https://docs.coingecko.com/reference/authentication\"}}"
}

→ getExchangeRate BTC→ETH
{
  "result": {
    "exchange_rate": 27.234714903164964,
    "from": "bitcoin",
    "to": "ethereum"
  }
}


In [ ]:
# FECTHING PRICE OF BTCUSD AND ETHUSD USING API ( COIN GECKO )

In [3]:
import requests
import pandas as pd
from datetime import datetime

API_KEY = "CG-fhdRS442CWoiXmvCUHwUixWW"   # your key
BASE = "https://api.coingecko.com/api/v3" # demo base URL
HEADERS = {"x-cg-pro-api-key": API_KEY}

def fetch_prices(symbols=["bitcoin", "ethereum"], vs="usd"):
    url = f"{BASE}/simple/price"
    params = {"ids": ",".join(symbols), "vs_currencies": vs}
    r = requests.get(url, params=params, headers=HEADERS)
    r.raise_for_status()
    return r.json()

data = fetch_prices(["bitcoin", "ethereum"], "usd")

# Convert to a clean table
rows = []
for sym, val in data.items():
    rows.append({
        "Symbol": sym.upper(),
        "Currency": "USD",
        "Price": val["usd"],
        "Timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    })

df = pd.DataFrame(rows)
print("🪙 Live Crypto Prices")
display(df)


🪙 Live Crypto Prices


,Symbol,Currency,Price,Timestamp
0,BITCOIN,USD,121865.00,2025-10-04 17:11:17
1,ETHEREUM,USD,4474.62,2025-10-04 17:11:17
